In [61]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [62]:
char_params = np.array([-111.1574, 3.9705, 0.0362, -7.1958, 3.0419, 1.2430, 0.1276, 1.1034, 1.4281])
methane_params = np.array([16.5264, 0.4982, -0.0304, 0.1739, 0.2148, -0.1864, -0.1186, -0.2192, -0.2552])

def char(ER, MC, T, C, H, O, ash):
    a = lambda i: char_params[i]
    return np.exp(a(0) + a(1)*ER + a(2)*MC + a(3)*((10**(-3))*T) + a(4)*((10**(-3))*T)**2 + a(5)*C + a(6)*H + a(7)*O +a(8)*ash)

def met(ER, MC, T, C, H, O, ash):
    b = lambda i: methane_params[i]
    return np.exp(b(0) + b(1)*ER + b(2)*MC + b(3)*((10**(-3))*T) + b(4)*((10**(-3))*T)**2 + b(5)*C + b(6)*H + b(7)*O +b(8)*ash)

In [63]:
ER = 0.27
MC = 0.075
species = ['H2', 'CO', 'CO2', 'CH4', 'N2']
exp = [16.8, 25.8, 9.1, 1.5, 46.8]

barrioWood = fs.create_fuel_stream(1, [50.70, 6.90, 42.40, 0.30, 0.00, 0.00], 0.0039, MC, 18.86)
air = fs.create_air_from_ER(barrioWood, ER)
outlet0 = gs.gasify_nonisot(barrioWood, air, charFormation=0, directMethaneConv=0)
Teq = outlet0.T
char1 = char(ER, MC, Teq, 50.7, 6.90, 42.40, 0.39)
met1 = met(ER, MC, Teq, 50.7, 6.90, 42.40, 0.39)
print(char1, met1)
outlet1 = gs.gasify_nonisot(barrioWood, air, charFormation=char1, directMethaneConv=0)
outlet2 = gs.gasify_nonisot(barrioWood, air, charFormation=0, directMethaneConv=met1)
outlet3 = gs.gasify_nonisot(barrioWood, air, charFormation=char1, directMethaneConv=met1)

for i, sp in enumerate(species):
    print(100*outlet3.get_syngas_fraction(sp, water=False, nitrogen=True))
print(outlet0.T, outlet3.T)
RMSE_0 = (sum([(exp[i] - 100*outlet0.get_syngas_fraction(sp, water=False, nitrogen=True))**2 for i, sp in enumerate(species)]) / len(species))**(1/2)
RMSE_1 = (sum([(exp[i] - 100*outlet1.get_syngas_fraction(sp, water=False, nitrogen=True))**2 for i, sp in enumerate(species)]) / len(species))**(1/2)
RMSE_2 = (sum([(exp[i] - 100*outlet2.get_syngas_fraction(sp, water=False, nitrogen=True))**2 for i, sp in enumerate(species)]) / len(species))**(1/2)
RMSE_3 = (sum([(exp[i] - 100*outlet3.get_syngas_fraction(sp, water=False, nitrogen=True))**2 for i, sp in enumerate(species)]) / len(species))**(1/2)
print(RMSE_0, RMSE_1, RMSE_2, RMSE_3)
print(100*outlet3.get_syngas_fraction('CH4', water=False, nitrogen=True))
# outlet2 = gs.gasify_nonisot2(barrioWood, air, charFormation=char1, directMethaneConv=met1)
# print(outlet2.T)
# print(outlet2.get_syngas_fraction('H2', water=False, nitrogen=True))
# print(outlet2.get_syngas_fraction('CO', water=False, nitrogen=True))
# print(outlet2.get_syngas_fraction('CO2', water=False, nitrogen=True))
# print(outlet2.get_syngas_fraction('CH4', water=False, nitrogen=True))
# print(outlet2.get_syngas_fraction('N2', water=False, nitrogen=True))

0.04736461979829722 0.08982715674014347
18.781254244492537
26.05928569366626
7.893473593643492
3.535229827259486
43.17776861347578
1306.6013594753942 1297.2668893001076
6.136925909460035 5.414137249707376 2.91915465680778 2.1312448985878225
3.535229827259486
